In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Configuração do matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Sklearn e Skforecast
from sklearn.ensemble import RandomForestRegressor
from skforecast.recursive import ForecasterRecursive
from skforecast.preprocessing import RollingFeatures

In [ ]:
dataset_path = '../../datasets/AEP_hourly.csv'

In [ ]:
df = pd.read_csv(dataset_path, delimiter=',')
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df = df.rename(columns={'Datetime': 'date', 'AEP_MW': 'y'})
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('date')
df = df.asfreq('1h', method='bfill', fill_value=0.0)
df = df.sort_index()

fig, ax = plt.subplots(figsize=(9, 4))
df['y'].plot(ax=ax, label='y')
df['exog_1'].plot(ax=ax, label='exogenous variable')
ax.legend()

In [ ]:
# Divide dados em treinamento e teste
# Para séries temporais, sempre utilizamos os dados mais recentes para teste
steps = 36
data_train = df[:-steps]
data_test  = df[-steps:]

print(f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

# Plotando os dados
fig, ax=plt.subplots(figsize=(9, 4))
data_train['y'].plot(ax=ax, label='train')
data_test['y'].plot(ax=ax, label='test')
ax.legend()

In [ ]:
forecaster = ForecasterRecursive(
                regressor = RandomForestRegressor(max_depth=10, n_estimators=50, random_state=123),
                lags      = 12,
                window_features = RollingFeatures(stats=['mean'], window_sizes=10)
             )

forecaster.fit(y=data_train['y'], exog=data_train['exog_1'])
forecaster

In [ ]:
# Faz predição no conjunto de teste
steps = 36
predictions = forecaster.predict(steps=steps, exog=data_test['exog_1'])
predictions.head(5)

In [ ]:
# Plota a predição, comparando com valores reais
fig, ax=plt.subplots(figsize=(9, 4))
data_train['y'].plot(ax=ax, label='train')
data_test['y'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predictions', color='green')
ax.legend()

In [ ]:
# import joblib

In [ ]:
# joblib.dump(model, 'house_price_linear_regression_model.pkl')